# Backtest Parameter Grid (Local Jupyter)
This notebook runs parameter grid explorations over the trading strategy using the existing backtest engine, **for local Jupyter**.
Steps:
1. Start Jupyter from the project root (where `src/` lives).
2. Open this notebook from the `notebooks/` directory.
3. Run the cells from top to bottom.

# Initialize

In [24]:
from pathlib import Path
import sys

# Assume this notebook lives in '<project_root>/notebooks'.
# Derive the project root as the parent directory and ensure it is on sys.path
# so that 'src' is importable without changing the process working directory.
project_root = Path().resolve().parent

if str(project_root) not in sys.path:
    sys.path.append(str(project_root))

print("Project root:", project_root)
print("Has src?:", (project_root / "src").exists())

Project root: C:\Users\Anton\SRC\my\ml_lstm
Has src?: True


In [25]:
import itertools
import os

import numpy as np
import pandas as pd

from src.backtest import (
    _compute_atr_series,
    _compute_backtest_metrics,
    _make_naive_prediction_provider,
    _make_model_prediction_provider,
    _load_predictions_csv,
    _make_csv_prediction_provider,
    run_backtest_on_dataframe,
)
from src.backtest_engine import BacktestConfig, run_backtest
from src.trading_strategy import StrategyConfig
from src.config import FREQUENCY, get_hourly_data_csv_path, get_predictions_csv_path

In [26]:
# Configuration for this notebook run
symbol = "nvda"
frequency = FREQUENCY  # Default from global config; override here if desired, e.g. "60min"
initial_equity = 10_000.0
prediction_mode = "model"  # "naive", "model", or "csv"

csv_path = get_hourly_data_csv_path(frequency)
print("Using OHLC data from:", csv_path)
data = pd.read_csv(csv_path)

# Basic sanity check
required_cols = {"Open", "High", "Low", "Close"}
missing = required_cols - set(data.columns)
if missing:
    raise ValueError(f"Data file {csv_path} is missing required columns: {missing}")

# Compute ATR(14) and a scalar ATR proxy
atr_series = _compute_atr_series(data, window=14)
atr_like = float(atr_series.dropna().mean()) if not atr_series.dropna().empty else 1.0
print(f"Mean ATR proxy: {atr_like:.4f}")

# Build prediction provider
if prediction_mode == "naive":
    provider = _make_naive_prediction_provider(offset_multiple=2.0, atr_like=atr_like)
elif prediction_mode == "model":
    provider = _make_model_prediction_provider(data, frequency=frequency)
elif prediction_mode == "csv":
    predictions_csv = get_predictions_csv_path(symbol, frequency)
    print("Using predictions CSV:", predictions_csv)
    preds_df = _load_predictions_csv(predictions_csv)
    provider = _make_csv_prediction_provider(preds_df, data)
else:
    raise ValueError(f"Unknown prediction_mode: {prediction_mode}")

Using OHLC data from: C:\Users\Anton\SRC\my\ml_lstm\data\processed\nvda_15min.csv
Mean ATR proxy: 0.3819
Using predictions CSV: C:\Users\Anton\SRC\my\ml_lstm\backtests\nvda_15min_predictions.csv


In [27]:
def run_one(
    strat_cfg: StrategyConfig,
    commission_per_unit_per_leg: float = 0.005,
    min_commission_per_order: float = 1.0,
):
    '''Run a single backtest with the given strategy and commission settings.'''
    bt_cfg = BacktestConfig(
        initial_equity=initial_equity,
        strategy_config=strat_cfg,
        model_error_sigma=atr_like,
        fixed_atr=atr_like,
        commission_per_unit_per_leg=commission_per_unit_per_leg,
        min_commission_per_order=min_commission_per_order,
    )

    result = run_backtest(
        data=data,
        prediction_provider=provider,
        cfg=bt_cfg,
        atr_series=atr_series,
        model_error_sigma_series=atr_series,
    )

    metrics = _compute_backtest_metrics(
        result,
        initial_equity=initial_equity,
        data=data,
    )

    row = {
        "final_equity": result.final_equity,
        "n_trades": len(result.trades),
        **metrics,
    }
    return result, row

# Grid A: risk_per_trade_pct × reward_risk_ratio

In [28]:
# Grid A: risk_per_trade_pct × reward_risk_ratio
risk_grid = [0.01, 0.02, 0.03]  # 0.0025, 0.005, 0.01, 0.03
rr_grid = [3.5, 4.0]  # [1.5, 2.0, 3.0]

rows = []
for risk_pct, rr in itertools.product(risk_grid, rr_grid):
    strat = StrategyConfig(
        risk_per_trade_pct=risk_pct,
        reward_risk_ratio=rr,
        k_sigma_err=0.5,
        k_atr_min_tp=3,
    )
    _, res_row = run_one(strat)
    res_row.update(
        {
            "grid": "risk_rr",
            "risk_per_trade_pct": risk_pct,
            "reward_risk_ratio": rr,
            "k_sigma_err": strat.k_sigma_err,
            "k_atr_min_tp": strat.k_atr_min_tp,
        }
    )
    rows.append(res_row)

df_risk_rr = pd.DataFrame(rows).sort_values("sharpe_ratio", ascending=False)
df_risk_rr.head(10)

,final_equity,n_trades,total_return,cagr,max_drawdown,sharpe_ratio,win_rate,profit_factor,grid,risk_per_trade_pct,reward_risk_ratio,k_sigma_err,k_atr_min_tp
3,2277.320990,1104,-0.772268,-0.399326,-0.862955,-0.201983,0.249094,0.942495,risk_rr,0.02,4.0,0.5,3
1,6083.102708,1104,-0.391690,-0.157377,-0.571369,-0.202544,0.249094,0.957905,risk_rr,0.01,4.0,0.5,3
5,503.534330,1104,-0.949647,-0.642842,-0.968873,-0.217982,0.249094,0.922776,risk_rr,0.03,4.0,0.5,3
0,4702.540512,1086,-0.529746,-0.228881,-0.617374,-0.493188,0.263352,0.928246,risk_rr,0.01,3.5,0.5,3
2,1467.162909,1086,-0.853284,-0.483751,-0.885879,-0.494689,0.263352,0.909949,risk_rr,0.02,3.5,0.5,3
4,240.136763,1086,-0.975986,-0.723253,-0.979008,-0.569677,0.263352,0.881016,risk_rr,0.03,3.5,0.5,3


# Grid B: model trust (k_sigma_err) × noise filter (k_atr_min_tp)

In [30]:
# Grid B: model trust (k_sigma_err) × noise filter (k_atr_min_tp)
k_sigma_grid = [0.25, 0.5, 0.75, 1.0]
k_atr_grid = [1.5, 2.0, 2.5, 3.0, 3.5]

rows = []
for k_sigma, k_atr_min_tp in itertools.product(k_sigma_grid, k_atr_grid):
    strat = StrategyConfig(
        risk_per_trade_pct=0.02,
        reward_risk_ratio=3.5,
        k_sigma_err=k_sigma,
        k_atr_min_tp=k_atr_min_tp,
    )
    _, res_row = run_one(strat)
    res_row.update(
        {
            "grid": "noise_filters",
            "risk_per_trade_pct": strat.risk_per_trade_pct,
            "reward_risk_ratio": strat.reward_risk_ratio,
            "k_sigma_err": k_sigma,
            "k_atr_min_tp": k_atr_min_tp,
        }
    )
    rows.append(res_row)

df_noise = pd.DataFrame(rows).sort_values("sharpe_ratio", ascending=False)
df_noise.head(10)

,final_equity,n_trades,total_return,cagr,max_drawdown,sharpe_ratio,win_rate,profit_factor,grid,risk_per_trade_pct,reward_risk_ratio,k_sigma_err,k_atr_min_tp
19,74844.786899,601,6.484479,1.000510,-0.430724,1.433099,0.337770,1.167385,noise_filters,0.02,3.5,1.00,3.5
14,70044.098049,666,6.004410,0.955342,-0.401130,1.361102,0.327327,1.174967,noise_filters,0.02,3.5,0.75,3.5
9,43244.388162,747,3.324439,0.656044,-0.556977,1.081532,0.310576,1.131634,noise_filters,0.02,3.5,0.50,3.5
18,29277.155103,762,1.927716,0.447821,-0.542661,0.878462,0.311024,1.082941,noise_filters,0.02,3.5,1.00,3.0
4,17546.828867,841,0.754683,0.213736,-0.671122,0.617133,0.291320,1.050121,noise_filters,0.02,3.5,0.25,3.5
13,12843.175226,896,0.284318,0.090025,-0.691323,0.468790,0.291295,1.012494,noise_filters,0.02,3.5,0.75,3.0
8,1467.162909,1086,-0.853284,-0.483751,-0.885879,-0.494689,0.263352,0.909949,noise_filters,0.02,3.5,0.50,3.0
3,1079.146240,1207,-0.892085,-0.535586,-0.919463,-0.560027,0.261806,0.842593,noise_filters,0.02,3.5,0.25,3.0
17,614.362220,1115,-0.938564,-0.617507,-0.953523,-0.865176,0.263677,0.893958,noise_filters,0.02,3.5,1.00,2.5
12,135.013005,1288,-0.986499,-0.773049,-0.988175,-1.388212,0.256211,0.825105,noise_filters,0.02,3.5,0.75,2.5


In [9]:
df_noise.head(30)

,final_equity,n_trades,total_return,cagr,max_drawdown,sharpe_ratio,win_rate,profit_factor,grid,risk_per_trade_pct,reward_risk_ratio,k_sigma_err,k_atr_min_tp
7,40483.777190,99,3.048378,0.623280,-0.161862,1.910401,0.393939,2.114007,noise_filters,0.02,3.5,0.50,2.5
2,37988.052895,115,2.798805,0.587887,-0.173406,1.761737,0.365217,1.907052,noise_filters,0.02,3.5,0.25,2.5
6,40183.099755,129,3.018310,0.619093,-0.161862,1.757574,0.356589,1.704803,noise_filters,0.02,3.5,0.50,2.0
16,36828.490264,112,2.682849,0.570924,-0.190078,1.741343,0.366071,1.858957,noise_filters,0.02,3.5,1.00,2.0
11,35754.413047,122,2.575441,0.554898,-0.173435,1.666348,0.352459,1.703386,noise_filters,0.02,3.5,0.75,2.0
3,29752.279762,97,1.975228,0.458988,-0.173406,1.577026,0.360825,1.828872,noise_filters,0.02,3.5,0.25,3.0
8,27306.858229,84,1.730686,0.416273,-0.144589,1.541677,0.369048,1.803601,noise_filters,0.02,3.5,0.50,3.0
12,29398.108500,106,1.939811,0.452947,-0.173435,1.521067,0.349057,1.690137,noise_filters,0.02,3.5,0.75,2.5
19,24625.875508,72,1.462588,0.366464,-0.173346,1.480319,0.375000,1.935685,noise_filters,0.02,3.5,1.00,3.5
9,22534.578455,72,1.253458,0.325089,-0.158212,1.362316,0.361111,1.731953,noise_filters,0.02,3.5,0.50,3.5


# Walk-forward backtests (configurable slices)

In [7]:
# Configure number of walk-forward slices over the available data
num_slices = 8  # change this to control how many test windows you want

if "Time" not in data.columns:
    raise ValueError("Walk-forward requires a 'Time' column in the OHLC data.")

time_series = pd.to_datetime(data['Time'])
n = len(data)
if n < num_slices:
    raise ValueError(f'Not enough bars ({n}) for num_slices={num_slices}')

boundaries = np.linspace(0, n, num_slices + 1, dtype=int)

walk_rows = []
for i in range(num_slices):
    start_idx, end_idx = int(boundaries[i]), int(boundaries[i + 1])
    if end_idx <= start_idx:
        continue
    df_slice = data.iloc[start_idx:end_idx].reset_index(drop=True)

    # Determine predictions CSV if using CSV-based predictions.
    preds_path = predictions_csv if 'predictions_csv' in globals() and prediction_mode == 'csv' else None

    result_slice = run_backtest_on_dataframe(
        df_slice,
        initial_equity=initial_equity,
        frequency=frequency,
        prediction_mode=prediction_mode,
        predictions_csv=preds_path,
    )

    metrics_slice = _compute_backtest_metrics(
        result_slice,
        initial_equity=initial_equity,
        data=df_slice,
    )

    start_time = time_series.iloc[start_idx]
    end_time = time_series.iloc[end_idx - 1]

    row = {
        'slice': i,
        'start_time': start_time,
        'end_time': end_time,
        'final_equity': result_slice.final_equity,
        'n_trades': len(result_slice.trades),
        **metrics_slice,
    }
    walk_rows.append(row)

df_walkforward = pd.DataFrame(walk_rows)
df_walkforward

,slice,start_time,end_time,final_equity,n_trades,total_return,cagr,max_drawdown,sharpe_ratio,win_rate,profit_factor
0,0,2022-12-30 11:00:00,2023-04-20 20:15:00,10322.915076,1,0.032292,0.109839,0.000000,1.810978,1.000000,0.000000
1,1,2023-04-20 20:30:00,2023-08-08 06:00:00,10693.500802,2,0.069350,0.250908,0.000000,2.351783,1.000000,0.000000
2,2,2023-08-08 06:15:00,2023-11-22 07:45:00,10205.446410,2,0.020545,0.072544,-0.020200,0.858397,0.500000,2.017061
3,3,2023-11-22 08:00:00,2024-04-09 20:30:00,11464.264298,21,0.146426,0.430085,-0.089589,1.528630,0.380952,1.497321
4,4,2024-04-09 20:45:00,2024-09-03 22:00:00,15016.993384,34,0.501699,1.745360,-0.040412,3.129371,0.470588,2.094570
5,5,2024-09-03 22:15:00,2025-01-24 02:45:00,11649.599970,6,0.164960,0.480264,-0.020454,2.447998,0.666667,4.711957
6,6,2025-01-24 11:00:00,2025-06-24 12:30:00,9892.706792,51,-0.010729,-0.025745,-0.133438,0.100290,0.294118,0.984801
7,7,2025-06-24 12:45:00,2025-11-18 17:15:00,10000.000000,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
